Done so far :


*   Lemmatization
*   Stop Words Removal

Verify :

* Normalization - removing accents, etc.
* Dates replaced with strings
* Case-folding
* Removed HTML entity codes



In [1]:
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import wordninja 

####### After importing nltk, run the following only once ######
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('stopwords')
### pip install wordninja ###

In [2]:
def remove_htmlcodes(document):
    
    replacement = {
                    "&ampnbsp": ' ',
                    "&ampamp": '&',
                    "&ampquot": '\'',
                    "&ampldquo": '\"',
                    "&amprdquo": '\"',
                    "&amplsquo": '\'',
                    "&amprsquo": '\'',
                    "&amphellip": '...',
                    "&ampndash": '-',
                    "&ampmdash": '-'
                  }
    
    for str in replacement:
        document = document.replace(str, replacement[str])
        
    return document

In [3]:
def get_wordnet_pos(word):
    
    tag=nltk.pos_tag([word])[0][1][0].upper()
    tag_dict={"J": wordnet.ADJ, 
              "N": wordnet.NOUN,
              "V": wordnet.VERB,
              "R": wordnet.ADV}
    return tag_dict.get(tag,wordnet.NOUN)

def lemma_stop(str):
    
    lemmatizer = WordNetLemmatizer()
    tokenizer = RegexpTokenizer('\w+|\$]\d\[+|\S+,-')
    tokenized = tokenizer.tokenize(str)
    lemmatized = [lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in tokenized]
    stop_words = set(stopwords.words('english'))
    filtered_sentence = [w for w in lemmatized if w.lower() not in stop_words]
    after_lemma_stop = ' '.join(w for w in filtered_sentence)
    return filtered_sentence

In [4]:
def is_not_credible (text):
    
    match = re.search(r'[!@#?&{}()]', text)
    
    if match:
        return True
    else:
        return False

In [5]:
def scrub_words(text):
    
    text = re.sub('[!@#?&{}()]', '', text)
    text=re.sub(r'[^\x00-\x7F]'," ",text)
    return text

In [6]:
def clean_document (document_string):
    
    cleaned_doc = document_string
    for word in document_string.split():
                if is_not_credible(word):
                    temp= scrub_words(word)
                    split=wordninja.split(temp)
                    if len(split)>7:
                          cleaned_doc = cleaned_doc.replace(word,'')
                    else:
                        replace_with=' '.join(word for word in split)
                        cleaned_doc = cleaned_doc.replace(word, replace_with)
    return cleaned_doc

In [8]:
from datetime import datetime

def replace_dates(documentString, docID):
    
    # regEx = '(([0-9]+(/|\\.|-)[0-9]+(/|\\.|-)[0-9]+)|([0-9]+(/|\\.|-)[0-9]+))'
    regEx = '(([0-9]+(/)[0-9]+(/)[0-9]+)|([0-9]+(/)[0-9]+))'
    iterator = re.finditer(regEx, documentString)
    listOfDates = [(m.start(0), m.end(0)) for m in iterator]
    tmp = []
    replace_with = []
    for indices in listOfDates:
        date = documentString[indices[0]:indices[1]]
        tmp.append(date)
        # date = date.replace('.', '/')
        # date = date.replace('-', '/')
        count = date.count('/')
        newDate = ''
        if count == 2:
            check_year = date[-3]
            
            if check_year == '/':
                YY = date[-2:]
                
                if int(YY) <= 19:
                    proper_date = date[:-2] + '20' + YY
                    date = date.replace(date,proper_date)
                else:
                    proper_date = date[:-2] + '19' + YY
                    date = date.replace(YY,('19'+YY))
                    
            try:
                newDate = datetime.strptime(date, '%m/%d/%Y').strftime('%d %b %Y')
            except ValueError as ve:
                newDate = date
        else:
            try:
                newDate = datetime.strptime(date, '%m/%d').strftime('%d %b')
            except ValueError as ve:
                newDate = date
        count_dates.append([docID, date])
        newDate = newDate.replace(' ', '')
        replace_with.append(newDate)
        
    for i in range(len(tmp)):
        documentString = documentString.replace(tmp[i], replace_with[i])
    
    return documentString

In [15]:
filet = "/home/nihaljain/3-1/CS F469/Assignment-1/mod_titles"
filec = "/home/nihaljain/3-1/CS F469/Assignment-1/mod_contents"
filed = "/home/nihaljain/3-1/CS F469/Assignment-1/mod_data"

titles = np.load(filet + ".npy", allow_pickle = True)
contents = np.load(filec + ".npy", allow_pickle = True)
data = np.load(filed + ".npy", allow_pickle = True)

In [16]:
import unidecode
from tqdm import tqdm_notebook

N = len(titles)

for i in tqdm_notebook(range(N)):
    for j in range(len(contents[i])):
        contents[i][j] = unidecode.unidecode(contents[i][j])
    for j in range(len(titles[i])):
        titles[i][j] = unidecode.unidecode(titles[i][j])

In [17]:
import pickle

with open('modified_contents_ascii.pickle', 'wb') as handle:
    pickle.dump(contents, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('modified_titles_ascii.pickle', 'wb') as handle:
    pickle.dump(titles, handle, protocol=pickle.HIGHEST_PROTOCOL)

# To read the data again

# with open('modified_contents_ascii.pickle', 'rb') as handle:
#     contents = pickle.load(handle)
# with open('modified_titles_ascii.pickle', 'rb') as handle:
#     titles = pickle.load(handle)
    
# print(unserialized_title == titles)

In [109]:
import trie
import pickle

N = 89163

getReference = {}

# contents = []
# titles = []
# data = []

# with open('modified_contents_ascii.pickle', 'rb') as handle:
#     contents = pickle.load(handle)
# with open('modified_titles_ascii.pickle', 'rb') as handle:
#     titles = pickle.load(handle)
# data = np.load("mod_data.npy", allow_pickle = True)

get_docID = {}
get_index = {}

for i in range(0, len(data)) :
    get_docID[i] = int(data[i][0])
    get_index[int(data[i][0])] = i

In [69]:
# for i in range(len(data)//2, len(data)//2 + 1000): print(get_docID[i])
print(get_index[127779])

102672


In [124]:
documentRoot = {}
collection = trie.CollectionNode()

# initializing the root for N documents
for i in range(len(data)//2, len(data)):
    newDocument = trie.Node()
    documentRoot[get_docID[i]] = newDocument

In [129]:
# creating the documents
import pickle

max_tf = {}
# contents = []
# titles = []
# data = []

# with open('modified_contents_ascii.pickle', 'rb') as handle:
#     contents = pickle.load(handle)
# with open('modified_titles_ascii.pickle', 'rb') as handle:
#     titles = pickle.load(handle)
# data = np.load("mod_data.npy", allow_pickle = True)

# N = len(contents)

import time
from tqdm import tqdm_notebook

start = time.time()
j = 0
for i in tqdm_notebook(range(len(data)//2, len(data)//2 + 1000)):
    if data[i][4] == None or data[i][1] == None or data[i][0] == None:
        continue
    for w in contents[j]:
        collection.add_document(w, 0, get_docID[i])
        documentRoot[get_docID[i]].add(w, 0)
        if get_docID[i] in max_tf:
            max_tf[get_docID[i]] = max(documentRoot[get_docID[i]].count_words(w, 0), max_tf[get_docID[i]])
        else:
            max_tf[get_docID[i]] = documentRoot[get_docID[i]].count_words(w, 0)
    for w in titles[j]:
        collection.add_title(w, 0, get_docID[i])
    j += 1
        
print(time.time() - start)  #39.19705152511597


2.947448968887329


In [48]:
with open('collection.pickle', 'wb') as handle:
    pickle.dump(collection, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('documentRoot.pickle', 'wb') as handle:
    pickle.dump(documentRoot, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# reading from pickle files

# with open('collection.pickle', 'rb') as handle:
#     collection = pickle.load(handle)
# with open('documentRoot.pickle', 'rb') as handle:
#     documentRoot = pickle.load(handle)

In [49]:
# import math
# import queue

# documentLength = {}
# N = len(documentRoot)

# for i in tqdm(range(len(documentRoot))):
    
#     docID = get_docID[i]
#     length = 0
#     document = documentRoot[i]
#     q = queue.Queue()
#     q.put([document, ''])

#     while q.qsize() > 0:

#         current = q.get()
#         reference = current[0]
#         word = current[1]

#         if reference.words > 0:
#             df = len(collection.get_doc_list(word, 0))
#             idf = math.log10(N/df)
#             # print(word, reference.words, df)
#             length += (reference.words * idf) ** 2

#         for i in range(256):
#             if reference.children[i] is not None:
#                 new_word = word + chr(i)
#                 q.put([reference.children[i], new_word])

#     # print(length**0.5)
#     documentLength[docID] = length**0.5

In [126]:
import unidecode

query = '9/11'
final_query = replace_dates(query, -1)
final_query = lemma_stop(final_query)

for i in range(len(final_query)):
    final_query[i] = unidecode.unidecode(final_query[i])
    # case-folding
    final_query[i] = final_query[i].lower()
print(final_query)

['11sep']


In [127]:
tf_query = {}
for w in final_query:
    if w not in tf_query:
        tf_query[w] = 1
    else:
        tf_query[w] += 1

***Ranked Retrieval based on TF-IDF Score :***


In [128]:
import queue

# scores[i] stores the dot product of the tf-idf score vectors of the query and document of docID i in the corpus
scores = {}
title_score = {}

# N is the total number of documents in the corpus
N = len(documentRoot)

# wordsInDoc[i] is a sorted list of (word, score) tuples where
# score is the tf-idf score for the (word, <ith doc>) pair
wordsInDoc = {}

factor = {}

import math
import bisect

for query_term in tf_query:
    
    docs_having_query_term = collection.get_doc_list(query_term, 0)
    df = len(docs_having_query_term)
    idf = 0
    
    print('-------------------------------------')
    print('Term in query = ', query_term)
    print()
    
    if df == 0:
        idf = 0
    else:
        idf = math.log10(N/df)
        
    docs_having_query_term_in_title = collection.get_title_list(query_term,0)
    
    for docID in docs_having_query_term_in_title:
        if docID in title_score:
            title_score[docID] += idf
        else:
            title_score[docID] = idf
        
    print('df = ',df)
    print('idf = ',idf)
    
    tfidf_query = tf_query[query_term] * idf
        
    for docID in docs_having_query_term:
        
        tf_doc = documentRoot[docID].count_words(query_term, 0)
        tf_doc = 0.5 + 0.5*tf_doc/max_tf[docID]
        tfidf_doc = (tf_doc)
        
        if docID not in scores:
            scores[docID] = (tfidf_query * tfidf_doc)
            wordsInDoc[docID] = []
            bisect.insort(wordsInDoc[docID], [-tfidf_query * tfidf_doc, query_term])
            factor[docID] = idf
        else:
            scores[docID] += (tfidf_query * tfidf_doc)
            bisect.insort(wordsInDoc[docID], [-tfidf_query * tfidf_doc, query_term])
            factor[docID] += idf
            
print(title_score)

for docID in scores:
    
    #if documentLength[docID] != 0:
    scores[docID] *= factor[docID]
    if docID in title_score:
        scores[docID] *= 1 + title_score[docID]

sorted_scores = sorted(scores.items(), key = lambda kv : kv[1] , reverse = True)

maxshow = min(10, len(scores))

print('\n\n')
print('============================================')

for i in range(maxshow):
    
    print()
    docID = sorted_scores[i][0]
    print('doc ID = ', docID)
    cnt = 0
    print('Keywords:')
    print()
    print(data[get_index[sorted_scores[i][0]]][1])
    print()
    if sorted_scores[i][0] not in title_score:
        print('title score = ',0)
    else:
        print('title score = ',title_score[sorted_scores[i][0]])
    for j in range(len(wordsInDoc[docID])):
        print(wordsInDoc[docID][j][1], wordsInDoc[docID][j][0], end = ' ')
        print(documentRoot[docID].count_words(wordsInDoc[docID][j][1], 0))
    print()
    print()
    count = 0
    found = 0
    words_before=queue.Queue()
    at_start = 1
    display = ""
    
    for word in data[get_index[docID]][4].split():
            
        check_with=replace_dates(word, -1)
        check_with = check_with.lower()
        if len(lemma_stop(check_with)) > 0:
            check_with=lemma_stop(check_with)[0]
        else:
            check_with=word
        
        if check_with == wordsInDoc[docID][0][1]:
            found=1
            
        if found == 1:
            display = display + word + " "
            count += 1
            if count == 50:
                break
        if found == 0:
            words_before.put(word)
            if words_before.qsize()>20:
                remove=words_before.get()
                at_start=0
                
    if not at_start:
        print('...', end = ' ')
    while words_before.qsize() > 0:
        print(words_before.get(), end = ' ')
    print(display, end = ' ')
    print('...', end = ' ')
    print('\n')
    print('tf-idf score=', sorted_scores[i][1])
    print('\n')
    print('============================================')

-------------------------------------
Term in query =  11sep

df =  29
idf =  3.5464916070505508
{127566: 3.5464916070505508, 131122: 3.5464916070505508, 129561: 3.5464916070505508, 128190: 3.5464916070505508, 127998: 3.5464916070505508}




doc ID =  131122
Keywords:

Daughter of only female cop killed on 9/11 stares down al Qaeda terrorist

title score =  3.5464916070505508
11sep -2.3643277380337 2


The daughter of fallen NYPD officer Moira Smith, who was the only female cop to die on 9/11, spent Mother’s Day in Guantanamo Bay — staring down one of the terrorists the US says helped kill her mom and more than 3,000 others. “I am glad Patricia was able to accompany me here to see for herself the monsters responsible for the murder of her mother,” the  ... 

tf-idf score= 38.122643465469395



doc ID =  127566
Keywords:

Families of 9/11 victims file suit against Saudi Arabia

title score =  3.5464916070505508
11sep -2.3643277380337 2


Families of 9/11 victims filed suit in Manhattan 

In [24]:
print((count_dates))

[[16, '50/50'], [42, '1/2'], [47, '4/4'], [47, '4/5'], [47, '3/4'], [47, '4/5'], [67, '24/7'], [70, '50/50'], [104, '9/11'], [151, '9/11'], [6432, '9/11'], [6440, '9/11'], [6492, '9/11'], [6620, '12/8'], [6701, '24/7'], [6765, '24/7'], [6802, '9/11'], [6811, '2008/8'], [6812, '12/22'], [6919, '1/5000'], [6919, '1/11'], [6919, '1/150'], [6919, '1/750'], [6919, '1/5000'], [6977, '9/11'], [6979, '1/2'], [6981, '6/19'], [7078, '9/11'], [7127, '9/11'], [7313, '24/7'], [-1, '2/5']]


In [118]:
print(collection.get_doc_list('jet',0))

{128067, 127788, 127440, 127411, 128116, 127157, 127611, 127260, 128190, 127359}


In [100]:
print(data[get_index[128190]][4])

FBI FBI FBI FBI FBI FBI FBI FBI FBI View Slideshow The FBI on Thursday   of the devastation from the 9/11 terror attack on the Pentagon. The 27 images show the gaping hole in the facade where a jet hijacked by Islamic jihadists slammed into the building, killing 184 innocents, including 125 Pentagon employees and 59 Other images show first responders battling stubborn blazes at the complex, debris from American Airlines Flight 77’s fuselage and haunting images from the scorched interior of the Arlington, Va. landmark. The airliner’s wings clipped off light poles before slamming into the western side of the building at the first-floor level. The jet ripped a hole into the structure that extended more than 100 yards into the building. It took firefighters days to fully extinguish the blaze. Luckily, the plane struck a portion of the building that was under renovation and relatively empty. If it had struck another side or happened after the renovations were completed, the death toll would

In [26]:
a = [['aaaa'],['bbbb']]
for document in a:
    document[0] = ''
    document = []
print(a)

[[''], ['']]
